# OLS, The Frisch-Waugh Theorem

This notebook illustrates the Frish-Waugh theorem.

In particular, it shows the following. First, we regress

$y = x_1'\beta_1 + x_2'\beta_2 + u$

Second, we run three regressions

1. $y = x_1'\gamma_1 + e_y$

2. $x_2 = x_1'\delta + e_2$

3. $e_y = e_2'\theta + v$,
where $(e_y,e_2)$ are from the regressions in 1. and 2.

Then, the estimates of $\beta_2$ and $\theta$ will be the same (as will their standard errors). This is used in, for instance, fixed effects panel regressions (where $x_1$ are dummies indicating different cross-sectional units).

## Load Packages and Extra Functions

The `OlsGM()` function was used in ch. 2. It is from the (local) `FinEcmt_OLS` module.

In [1]:
MyModulePath = joinpath(pwd(),"src")
!in(MyModulePath,LOAD_PATH) && push!(LOAD_PATH,MyModulePath);

In [2]:
using FinEcmt_OLS, DelimitedFiles, LinearAlgebra

## Loading Data

In [3]:
x  = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)
(Rme,SMB,HML,Rf) = (x[:,2],x[:,3],x[:,4],x[:,5])

x  = readdlm("Data/FF25Ps.csv",',') #no header line
R  = x[:,2:end]                     #returns for 25 FF portfolios
Re = R .- Rf                        #excess returns for the 25 FF portfolios

T = size(Re,1)                      #number of observations

388

In [4]:
y  = Re[:,6]                        #to the notation used in comment, use asset 6
x₁ = [ones(T) Rme]                  #1st set of regressors (2)
x₂ = [SMB HML];                     #2nd set of regressors

## Regress y on Both x₁ and x₂

In [5]:
(b,_,_,V,) = OlsGM(y,[x₁ x₂])
std_b = sqrt.(diag(V))

printblue("OLS Results from y regressed on x:\n")
rowNames=["x₁ (c)","x₁ (Rme)","x₂ (SMB)","x₂ (HML)"]
printmat([b std_b];colNames=["b","std"],rowNames)

OLS Results from y regressed on x:

                 b       std
x₁ (c)      -0.337     0.121
x₁ (Rme)     1.184     0.028
x₂ (SMB)     0.916     0.040
x₂ (HML)    -0.384     0.042



## The Three Steps in Frisch-Waugh

1. Regress `y` on `x₁`  and save the residuals as `e_y`. (Sorry, cannot create a symbol like $e_y$.)

2. Regress `x₂` on `x₁` and save the residuals as `e₂`.

3. Regress `e_y` on `e₂`.

In [6]:
(_,e_y,) = OlsGM(y,x₁);              #step 1

(_,e₂,) = OlsGM(x₂,x₁);              #step 2

(b,_,_,V,) = OlsGM(e_y,e₂)           #step 3
std_b = isa(V,Number) ? sqrt(V) : sqrt.(diag(V)) #diag() fails if V is a number (not a matrix)

printblue("OLS Results from e_y regressed on e₂:\n")
printmat([b std_b],colNames=["b","std"],rowNames=["e₂ (SMB)","e₂ (HML)"])
printred("Should be same coeff and std as in multiple regression (above)")

OLS Results from e_y regressed on e₂:

                 b       std
e₂ (SMB)     0.916     0.040
e₂ (HML)    -0.384     0.042

Should be same coeff and std as in multiple regression (above)


## A Partial Frisch-Waugh Approach (extra)

Regress `y` (not `e_y`) on `e₂`. This gives the same point estimate, but wrong standard error.

In [7]:
(b,_,_,V,) = OlsGM(y,e₂)             #step 3, adjusted
std_b = isa(V,Number) ? sqrt(V) : sqrt.(diag(V)) #diag() fails if V is a number (not a matrix)

printblue("OLS Results from y regressed on e₂:\n")
printmat([b std_b],colNames=["b","std"],rowNames=["e₂ (SMB)","e₂ (HML)"])
printred("Should be same coeff (but different std) as in multiple regression (above)")

OLS Results from y regressed on e₂:

                 b       std
e₂ (SMB)     0.916     0.120
e₂ (HML)    -0.384     0.124

Should be same coeff (but different std) as in multiple regression (above)
